<a href="https://colab.research.google.com/github/sdewhitt/Market-Anomaly-Detection/blob/main/Financial_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pandas.errors import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from google.colab import userdata
! pip install groq --quiet
from groq import Groq
client = Groq(
  api_key=userdata.get("GROQ_API_KEY")
)


import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 1.8 MB/s eta 0:00:00


#**Load data sheets**

In [2]:
data_path = '/content/drive/MyDrive/Colab Notebooks/FinancialMarketData.xlsx'
download = pd.read_excel(data_path, sheet_name="Download")
metadata = pd.read_excel(data_path, sheet_name="Metadata")
ews = pd.read_excel(data_path, sheet_name="EWS")
ews.rename(columns={'Data': 'Date'}, inplace=True)

In [ ]:
download

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59
0,EURUSDV3M Index,NaN,NaN,"Fixed Income, MM & Interbank",NaN,NaN,NaN,NaN,NaN,NaN,...,Treasury,Gold,Brent,S&P,Eurostoxx,Nasdaq,EM,Japan,2Y Ger,2Y US
1,NaN,NaN,Show/Hide data,S,H,H,H,H,H,H,...,H,H,H,H,H,H,H,H,H,H
2,676,NaN,Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Index Currency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Ticker,XAU BGNL Curncy,ECSURPUS Index,BDIY Index,CRY Index,DXY Curncy,JPY Curncy,GBP Curncy,...,TY1 Comdty,GC1 Comdty,CO1 Comdty,ES1 Index,VG1 Index,NQ1 Index,LLL1 Index,TP1 Index,DU1 Comdty,TU2 Comdty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,NaN,NaN,2021-03-23 00:00:00,1727.96,0.339,2271,184.5022,92.336,108.72,1.376,...,131.921875,1725.1,60.79,3899.8,3773,13006.25,1103.6,1958.5,112.14,110.382812
1150,NaN,NaN,2021-03-30 00:00:00,1685.56,0.303,2103,184.8741,93.297,110.29,1.3719,...,131.1875,1683.9,64.14,3947.7,3872,12878.25,1103.6,1978,112.09,110.386719
1151,NaN,NaN,2021-04-06 00:00:00,1743.28,0.44,2092,185.8393,92.335,109.88,1.3822,...,131.6875,1741.5,62.74,4064,3920,13570,1103.6,1955,112.12,110.28125
1152,NaN,NaN,2021-04-13 00:00:00,1742.69,0.467,2140,188.1368,91.852,109.19,1.3737,...,132.078125,1746.2,63.67,4132.8,3917,13975.75,1103.6,1961.5,112.115,110.292969


In [4]:
metadata

,In order to understand the business nature of these data:,Unnamed: 1,Unnamed: 2
0,1) Look below Table 1 for the features,NaN,NaN
1,"2) Alternatively/in addition, Google the strin...",NaN,NaN
2,NaN,NaN,NaN
3,Note,NaN,NaN
4,"What you find in the sheet ""DownloadBLB"" is a ...",NaN,NaN
5,NaN,NaN,NaN
6,Response variable,NaN,NaN
7,Y,"manually tagged: Y=1 if abnormal risk/crisis, ...",NaN
8,Features,NaN,NaN
9,Table 1,NaN,NaN


In [5]:
ews

,Y,Date,XAU BGNL,ECSURPUS,BDIY,CRY,DXY,JPY,GBP,Cl1,...,LP01TREU,EMUSTRUU,LF94TRUU,MXUS,MXEU,MXJP,MXBR,MXRU,MXIN,MXCN
0,0,2000-01-11,283.25,0.077,1388,157.2600,100.560,105.86,1.6460,25.77,...,116.4635,230.5267,123.7616,1416.12,127.75,990.59,856.76,224.33,217.34,34.30
1,0,2000-01-18,287.65,0.043,1405,165.0100,101.860,105.47,1.6383,28.85,...,117.2674,231.3770,123.7616,1428.79,129.50,993.98,925.22,234.37,227.08,32.74
2,0,2000-01-25,287.15,0.135,1368,167.2400,102.410,106.04,1.6496,28.28,...,117.9946,232.3895,123.7616,1385.93,126.48,974.83,886.93,216.82,233.00,32.46
3,0,2000-02-01,282.75,0.191,1311,166.8500,104.920,107.85,1.6106,28.22,...,120.5100,231.9417,122.3281,1385.31,129.19,1007.12,842.60,201.89,237.48,31.29
4,1,2000-02-08,298.40,0.312,1277,165.4300,104.220,109.30,1.6108,28.02,...,118.7914,237.8117,122.3281,1411.95,134.67,1034.58,945.15,218.00,258.02,31.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0,2021-03-23,1727.96,0.339,2271,184.5022,92.336,108.72,1.3760,57.76,...,427.7972,1249.4030,389.8500,3800.77,140.32,1199.57,1671.73,681.21,1695.96,110.83
1107,0,2021-03-30,1685.56,0.303,2103,184.8741,93.297,110.29,1.3719,60.55,...,428.6578,1242.2640,385.8145,3835.96,142.76,1206.29,1635.57,693.38,1690.76,108.80
1108,0,2021-04-06,1743.28,0.440,2092,185.8393,92.335,109.88,1.3822,59.33,...,430.6649,1250.2560,390.6819,3955.16,144.13,1194.62,1689.62,673.20,1689.48,111.92
1109,0,2021-04-13,1742.69,0.467,2140,188.1368,91.852,109.19,1.3737,60.18,...,430.4087,1252.5630,390.3848,4027.05,144.23,1197.06,1680.82,680.23,1672.46,108.40


In [8]:
ews.columns

Index(['Y', 'Date', 'XAU BGNL', 'ECSURPUS', 'BDIY', 'CRY', 'DXY', 'JPY', 'GBP',
       'Cl1', 'VIX', 'USGG30YR', 'GT10', 'USGG2YR', 'USGG3M', 'US0001M',
       'GTDEM30Y', 'GTDEM10Y', 'GTDEM2Y', 'EONIA', 'GTITL30YR', 'GTITL10YR',
       'GTITL2YR', 'GTJPY30YR', 'GTJPY10YR', 'GTJPY2YR', 'GTGBP30Y',
       'GTGBP20Y', 'GTGBP2Y', 'LUMSTRUU', 'LMBITR', 'LUACTRUU', 'LF98TRUU',
       'LG30TRUU', 'LP01TREU', 'EMUSTRUU', 'LF94TRUU', 'MXUS', 'MXEU', 'MXJP',
       'MXBR', 'MXRU', 'MXIN', 'MXCN'],
      dtype='object')

# **Find Appropriate Features**

In [ ]:
system_prompt = f"""
You are a machine learning + financial expert that is helping me work on building a financial anomaly detection model.
I need to understand the data that was provided to me.
Please tell me the most important features, bonds, financial indicators, etc. for the anomaly detection model based on the given data.
"""

prompt = f'EWS sheet: {ews.head(30).to_string()}'

llm_response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
)

response = llm_response.choices[0].message.content
#print(response)

### Llama 3.1 suggested features

Based on the provided data, here are the most important features, stocks, bonds, financial indicators, etc. for the anomaly detection model:

### Features:

1. **DXY (US Dollar Index)**: This is a benchmark index that measures the performance of the United States dollar against a basket of foreign currencies.
2. **VIX (Chicago Board Options Exchange Volatility Index)**: This is a measure of market volatility, often referred to as the "fear index".
3. **USGG30YR (30-Year US Treasury Bond Yield)**: This is a benchmark interest rate that reflects the yield on 30-year US Treasury bonds.
4. **USGG2YR (2-Year US Treasury Bond Yield)**: This is another benchmark interest rate that reflects the yield on 2-year US Treasury bonds.
5. **EC SURPUS (European Central Bank Surplus)**: This is a measure of the European Central Bank's surplus, which can impact market sentiment and interest rates.
6. **XAU (Gold Price)**: This is the price of gold, often considered a safe-haven asset and a hedge against inflation.
7. **BDIY (Baltic Dry Index)**: This is a measure of the price of shipping dry bulk commodities, which can impact global trade and economic activity.
8. **JPY (Japanese Yen Exchange Rate)**: This is the exchange rate between the US dollar and the Japanese yen, which can impact trade and investment between the two countries.
9. **GBP (British Pound Exchange Rate)**: This is the exchange rate between the US dollar and the British pound, which can impact trade and investment between the two countries.
10. **CRY (Credit Index)**: This is a measure of the creditworthiness of companies, which can impact market sentiment and interest rates.

### Stocks:

None are explicitly mentioned in the data, but the following indices can be used as proxies for stock market performance:

1. **MXUS (MSCI US Index)**: This is a benchmark index that tracks the performance of the US stock market.
2. **MXEU (MSCI Europe Index)**: This is a benchmark index that tracks the performance of the European stock market.
3. **MXJP (MSCI Japan Index)**: This is a benchmark index that tracks the performance of the Japanese stock market.
4. **MXBR (MSCI Brazil Index)**: This is a benchmark index that tracks the performance of the Brazilian stock market.
5. **MXRU (MSCI Russia Index)**: This is a benchmark index that tracks the performance of the Russian stock market.
6. **MXIN (MSCI India Index)**: This is a benchmark index that tracks the performance of the Indian stock market.
7. **MXCN (MSCI China Index)**: This is a benchmark index that tracks the performance of the Chinese stock market.

### Bonds:

1. **USGG30YR (30-Year US Treasury Bond Yield)**: This is a benchmark interest rate that reflects the yield on 30-year US Treasury bonds.
2. **USGG2YR (2-Year US Treasury Bond Yield)**: This is another benchmark interest rate that reflects the yield on 2-year US Treasury bonds.
3. **GT10 (10-Year German Bund Yield)**: This is a benchmark interest rate that reflects the yield on 10-year German government bonds.
4. **GTDEM30Y (30-Year German Bund Yield)**: This is a benchmark interest rate that reflects the yield on 30-year German government bonds.
5. **GTDEM10Y (10-Year German Bund Yield)**: This is a benchmark interest rate that reflects the yield on 10-year German government bonds.
6. **GTDEM2Y (2-Year German Bund Yield)**: This is a benchmark interest rate that reflects the yield on 2-year German government bonds.
7. **GTITL30YR (30-Year Italian Bund Yield)**: This is a benchmark interest rate that reflects the yield on 30-year Italian government bonds.
8. **GTITL10YR (10-Year Italian Bund Yield)**: This is a benchmark interest rate that reflects the yield on 10-year Italian government bonds.
9. **GTITL2YR (2-Year Italian Bund Yield)**: This is a benchmark interest rate that reflects the yield on 2-year Italian government bonds.
10. **GTJPY30YR (30-Year Japanese Government Bond Yield)**: This is a benchmark interest rate that reflects the yield on 30-year Japanese government bonds.
11. **GTJPY10YR (10-Year Japanese Government Bond Yield)**: This is a benchmark interest rate that reflects the yield on 10-year Japanese government bonds.
12. **GTJPY2YR (2-Year Japanese Government Bond Yield)**: This is a benchmark interest rate that reflects the yield on 2-year Japanese government bonds.
13. **GTGBP30Y (30-Year UK Gilt Yield)**: This is a benchmark interest rate that reflects the yield on 30-year UK government bonds.
14. **GTGBP20Y (20-Year UK Gilt Yield)**: This is a benchmark interest rate that reflects the yield on 20-year UK government bonds.
15. **GTGBP2Y (2-Year UK Gilt Yield)**: This is a benchmark interest rate that reflects the yield on 2-year UK government bonds.

### Other:

1. **EONIA (Euro OverNight Index Average)**: This is a benchmark interest rate that reflects the overnight rate in the eurozone.
2. **LUMSTRUU (Lumber Price)**: This is the price of lumber, often used as a proxy for the housing market and construction activity.
3. **LMBITR (Lumber Bit Rate)**: This is a measure of the bit rate used in lumber production, which can impact the efficiency of the industry.
4. **LUACTRUU (Lumber Actual Rate)**: This is a measure of the actual rate of lumber production, which can impact the supply of lumber.
5. **LF98TRUU (Lumber Futures Rate)**: This is a measure of the futures rate for lumber, which can impact market sentiment and prices.
6. **LG30TRUU (Lumber 30-Day Rate)**: This is a measure of the 30-day rate for lumber, which can impact market sentiment and prices.
7. **LP01TREU (Lumber 1-Year Rate)**: This is a measure of the 1-year rate for lumber, which can impact market sentiment and prices.
8. **EMUSTRUU (EMU Striking Rate)**: This is a measure of the striking rate in the eurozone, which can impact market sentiment and prices.

# **Model Training**

In [12]:
features = [
    'VIX',
    'DXY',
    'BDIY',
    'LUMSTRUU',
    'USGG30YR',
    'GT10',
    'GTDEM10Y',
    'GTITL10YR',
    'GTJPY10YR'
]

#features = ['VIX', 'DXY', 'USGG30YR', 'GT10']

focused_features = ews[['Y', 'Date'] + features]

window_count = 4 # each window is about a month long. TODO: try different window sizes
for f in features: # moving averages
  focused_features.loc[:, f'{f}_MA'] = focused_features[f].rolling(window=window_count).mean()

  # patch up NaN values with smaller temp windows until the intended window size is met
  for i in range(window_count - 1):
    focused_features.loc[i, f'{f}_MA'] = focused_features.loc[i, f]
    focused_features.loc[i, f'{f}_MA'] = focused_features.loc[:i, f'{f}_MA'].mean()
focused_features

,Y,Date,VIX,DXY,BDIY,LUMSTRUU,USGG30YR,GT10,GTDEM10Y,GTITL10YR,GTJPY10YR,VIX_MA,DXY_MA,BDIY_MA,LUMSTRUU_MA,USGG30YR_MA,GT10_MA,GTDEM10Y_MA,GTITL10YR_MA,GTJPY10YR_MA
0,0,2000-01-11,22.50,100.560,1388,870.94,6.6710,6.657,5.541,5.777,1.709,22.500000,100.560000,1388.000000,870.9400,6.671000,6.657000,5.54100,5.77700,1.709000
1,0,2000-01-18,21.50,101.860,1405,869.33,6.7470,6.748,5.644,5.889,1.765,22.000000,101.210000,1396.500000,870.1350,6.709000,6.702500,5.59250,5.83300,1.737000
2,0,2000-01-25,23.02,102.410,1368,873.50,6.6340,6.692,5.515,5.760,1.625,22.506667,101.393333,1384.166667,871.5250,6.671333,6.683833,5.54950,5.79000,1.690333
3,0,2000-02-01,23.45,104.920,1311,871.22,6.4230,6.619,5.459,5.710,1.770,22.617500,102.437500,1368.000000,871.2475,6.618750,6.679000,5.53975,5.78400,1.717250
4,1,2000-02-08,21.25,104.220,1277,872.39,6.2310,6.613,5.482,5.728,1.919,22.305000,103.352500,1340.250000,871.6100,6.508750,6.668000,5.52500,5.77175,1.769750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0,2021-03-23,20.30,92.336,2271,2301.89,2.3264,1.622,-0.343,0.601,0.077,22.055000,91.736000,1965.500000,2306.4975,2.282425,1.540750,-0.33350,0.64750,0.105250
1107,0,2021-03-30,19.61,93.297,2103,2297.31,2.3687,1.706,-0.288,0.679,0.085,20.932500,92.364000,2073.000000,2302.2325,2.326775,1.619000,-0.31725,0.64875,0.095750
1108,0,2021-04-06,18.12,92.335,2092,2302.52,2.3230,1.658,-0.317,0.691,0.108,19.455000,92.458250,2120.750000,2300.9325,2.349150,1.651500,-0.32125,0.64875,0.092500
1109,0,2021-04-13,16.65,91.852,2140,2309.15,2.2939,1.616,-0.293,0.748,0.101,18.670000,92.455000,2151.500000,2302.7175,2.328000,1.650500,-0.31025,0.67975,0.092750


### Isolation Forest with SMOTE





In [6]:
X = focused_features.iloc[:, 2:]
y_true = focused_features.iloc[:, 0] # 'Y' column
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE # trying to over sample for 1s

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y_true)

X_scaled = X_resampled
y_true = y_resampled

# Train Isolation Forest 0.213
model = IsolationForest(contamination=0.351, random_state=42)
model.fit(X_scaled) # use X_resampled if doing SMOTE

# Predict anomalies
anomaly_scores = model.decision_function(X_scaled)  # Anomaly scores (higher = normal, lower = anomalous)
anomaly_labels = model.predict(X_scaled)            # Predicted labels (-1 = anomaly, 1 = normal)

# Adjust labels to match labeling convention (1 = anomaly, 0 = normal)
anomaly_labels = [0 if label == 1 else 1 for label in anomaly_labels]


# Evaluate the model (requires ground truth labels `y_true`)
print(classification_report(y_true, anomaly_labels))
roc_auc = roc_auc_score(y_true, anomaly_scores)
print(f"ROC-AUC Score: {roc_auc}")

KeyboardInterrupt: 

### XGBoost with SMOTE

In [4]:
!pip install xgboost==1.7.6 --quiet
import xgboost as xgb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.8 MB/s eta 0:00:00


In [13]:
X = focused_features.iloc[:, 2:]
y_true = focused_features.iloc[:, 0] # 'Y' column

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_true, test_size=0.25, random_state=42, stratify=y_true)

scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

X_train = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


model = xgb.XGBClassifier(scale_pos_weight=scale_pos_weight, random_state=42)
model.fit(X_train, y_train)

# Predict anomalies
anomaly_scores = model.predict_proba(X_test_scaled)[:, 1]  # Anomaly Probability
anomaly_labels = model.predict(X_test_scaled)              # Predicted labels (0 or 1)


from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(y_test, anomaly_labels))
roc_auc = roc_auc_score(y_test, anomaly_scores)

from sklearn.metrics import precision_recall_curve, auc
precision, recall, thresholds = precision_recall_curve(y_test, anomaly_scores)
pr_auc = auc(recall, precision)
print(f"PR-AUC Score: {pr_auc}")

print(f"ROC-AUC Score: {roc_auc}")

              precision    recall  f1-score   support

           0       0.98      0.95      0.97       219
           1       0.85      0.93      0.89        59

    accuracy                           0.95       278
   macro avg       0.91      0.94      0.93       278
weighted avg       0.95      0.95      0.95       278

PR-AUC Score: 0.8896952476676848
ROC-AUC Score: 0.975853262131414


In [164]:
import joblib
joblib.dump(model, 'xgboost_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [84]:
test_size_values = np.linspace(0.1, 0.4, 20)  # Test sizes from 0.1 to 0.4 in 10 steps

results = []
x = focused_features.iloc[:, 2:]
y = focused_features.iloc[:, 0] # 'Y' column
for test_size in test_size_values:
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )

    # Train the model
    model = xgb.XGBClassifier(scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(), random_state=42)
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Evaluate metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Save results
    results.append({
        "test_size": test_size,
        "precision": report["1"]["precision"],
        "recall": report["1"]["recall"],
        "f1_score": report["1"]["f1-score"],
        "roc_auc": roc_auc
    })

# Find the best test_size based on a target metric (e.g., F1-score or ROC-AUC)
best_result = max(results, key=lambda x: x["roc_auc"])  # Change to 'f1_score' if needed
print("Best test_size:", best_result["test_size"])
print("Metrics:", best_result)

Best test_size: 0.1
Metrics: {'test_size': 0.1, 'precision': 0.8, 'recall': 0.8333333333333334, 'f1_score': 0.8163265306122449, 'roc_auc': 0.9758522727272727}


# PCA

In [1]:
X = focused_features.iloc[:, 2:21].values
y = focused_features.iloc[:, 0].values
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

pca = PCA(n_components=2)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

explained_variance = pca.explained_variance_ratio_

classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)

# Predicting the test set result using
# predict function under LogisticRegression
y_pred = classifier.predict(X_test)

# making confusion matrix between
#  test set of Y and predicted value.
cm = confusion_matrix(y_test, y_pred)

# Predicting the training set
# result through scatter plot
from matplotlib.colors import ListedColormap

X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start=X_set[:, 0].min() - 1,
                               stop=X_set[:, 0].max() + 1, step=0.01),
                     np.arange(start=X_set[:, 1].min() - 1,
                               stop=X_set[:, 1].max() + 1, step=0.01))

plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(),
                                                  X2.ravel()]).T).reshape(X1.shape), alpha=0.75,
             cmap=ListedColormap(('yellow', 'white', 'aquamarine')))

plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())

for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                color=ListedColormap(('red', 'green', 'blue'))(i), label=j)

plt.title('Logistic Regression')
plt.xlabel('PC1')  # for Xlabel
plt.ylabel('PC2')  # for Ylabel
plt.legend()  # to show legend

# show scatter plot
plt.show()


NameError: name 'focused_features' is not defined

In [111]:
#do_PCA(features[0])

# **yfinance testing**

In [30]:
import yfinance as yf
from datetime import datetime

# Define a list of tickers corresponding to requested features
tickers = {
    "VIX": "^VIX",  # Volatility Index
    "DXY": "DX-Y.NYB",  # US Dollar Index
    "USGG30YR": "^TYX",  # US 30-Year Treasury Yield
    "GT10": "^TNX",  # US 10-Year Treasury Yield
}

start_date = "2000-01-01"
end_date = "2024-12-31"


all_data = pd.DataFrame()

# Fetch data from Yahoo Finance
for label, ticker in tickers.items():
    if ticker:
        print(f"Fetching data for {label} ({ticker})...")
        ticker_data = yf.download(ticker, start=start_date, end=end_date)["Close"]  # Use Adjusted Close price
        ticker_data.name = label  # Rename the series to match the label
        all_data = pd.concat([all_data, ticker_data], axis=1)  # Merge into the main DataFrame
    else:
        print(f"Ticker for {label} is not available on yfinance.")


all_data.rename(columns={value: key for key, value in tickers.items()}, inplace=True)


# Save the time series data to CSV for further use
all_data.to_csv("yfinance_time_series.csv")

print("Data fetching complete. Saved to 'yfinance_time_series.csv'.")

# Resample to weekly frequency and aggregate using mean
all_data_weekly = all_data.resample('W').mean()

# Save the weekly data to CSV
all_data_weekly.to_csv("yfinance_time_series_weekly.csv")

print("Weekly data saved to 'yfinance_time_series_weekly.csv'.")

all_data_weekly



[*********************100%***********************]  1 of 1 completed

Fetching data for VIX (^VIX)...
Fetching data for DXY (DX-Y.NYB)...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetching data for USGG30YR (^TYX)...
Fetching data for GT10 (^TNX)...


[*********************100%***********************]  1 of 1 completed


Data fetching complete. Saved to 'yfinance_time_series.csv'.
Weekly data saved to 'yfinance_time_series_weekly.csv'.


Ticker,VIX,DXY,USGG30YR,GT10
Date,,,,
2000-01-09,25.016000,100.492001,6.5740,6.5370
2000-01-16,21.684000,100.978000,6.6590,6.6420
2000-01-23,21.447500,101.758002,6.7235,6.7505
2000-01-30,23.960000,102.940001,6.5646,6.6762
2000-02-06,23.014001,104.586000,6.3232,6.5748
...,...,...,...,...
2024-12-08,13.280000,106.167999,4.3538,4.1860
2024-12-15,13.936000,106.644000,4.4878,4.2828
2024-12-22,20.126000,107.576001,4.6604,4.4740
